# ДЗ  
1. Доработать паука в имеющемся проекте, чтобы он формировал item по структуре:
  * Наименование вакансии
  * Зарплата от
  * Зарплата до
  * Ссылку на саму вакансию

 и складывал все записи в БД.  
 

 2. Создать в имеющемся проекте второго паука по сбору вакансий с сайта rabota.ru. Паук должен формировать item'ы по аналогичной структуре и складывать данные также в БД.



# 1. Разархивируем данные

In [ ]:
!unzip scrapy.zip

In [2]:
%cd scrapy

/content/scrapy


# 2. Установкка Mongo

In [3]:
!apt install mongodb >log

In [4]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [5]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 3. Установка scrapy

In [ ]:
!pip install scrapy

# 4. Запуск паука

In [7]:
!scrapy crawl hh_ru

2023-01-11 20:08:16 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: parser_job)
2023-01-11 20:08:16 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.16 (default, Dec  7 2022, 01:12:13) - [GCC 7.5.0], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Linux-5.10.147+-x86_64-with-glibc2.27
2023-01-11 20:08:16 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'parser_job',
 'DOWNLOAD_DELAY': 0.5,
 'NEWSPIDER_MODULE': 'parser_job.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'SPIDER_MODULES': ['parser_job.spiders'],
 'USER_AGENT': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile '
               'Safari/537.36'}
2023-01-11 20:08:16 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2023-01-11 20:08:16 [scrapy.extensions.telnet] INFO: Tel

In [19]:
!scrapy crawl rabota_ru

2023-01-11 20:40:14 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: parser_job)
2023-01-11 20:40:14 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.16 (default, Dec  7 2022, 01:12:13) - [GCC 7.5.0], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Linux-5.10.147+-x86_64-with-glibc2.27
2023-01-11 20:40:14 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'parser_job',
 'DOWNLOAD_DELAY': 0.5,
 'NEWSPIDER_MODULE': 'parser_job.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'SPIDER_MODULES': ['parser_job.spiders'],
 'USER_AGENT': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile '
               'Safari/537.36'}
2023-01-11 20:40:14 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2023-01-11 20:40:14 [scrapy.extensions.telnet] INFO: Tel

# 5. Проверка, что записалось в Mongo

In [20]:
from pymongo import MongoClient
client = MongoClient('127.0.0.1', 27017)
print(client.list_database_names())
db = client['db_vacancies']
print(db.list_collection_names())

['admin', 'config', 'db_vacancies', 'local']
['hh_ru', 'rabota_ru']


# Запись в json

In [46]:
import json

class create_dict(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 
        
mydict = create_dict()
i = 1
for mycol in db.list_collection_names():
  for y in db[mycol].find():
    mydict.add(i, ({"company_name'":y['company_name'],"vacancy_name":y['name'],"salary":y['salary'], "url": y['url']}))
    i = i+1
  f_name = f'{mycol}_json'
  print(f_name)
  with open(f_name, 'w', encoding='UTF-8') as f:
          json.dump(mydict, f)
  # f_name = json.dumps(mydict, indent=2, sort_keys=True)


hh_ru_json
rabota_ru_json


In [49]:
with open('hh_ru_json', 'r', encoding='UTF-8') as f:
    HH = json.load(f)
print(f'hh_ru - количество вакансий в файле: {len(HH)}\n')
with open('rabota_ru_json', 'r', encoding='UTF-8') as f:
    RR = json.load(f)
print(f'Rabota.ru - количество вакансий в файле: {len(RR)}\n')

hh_ru - количество вакансий в файле: 30

Rabota.ru - количество вакансий в файле: 405

